# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [24]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [25]:
# Load the csv 
output_file = "../cities_output_student.csv"

# Create DataFrame for output data file from cities/weather (Part I)
cities_df = pd.read_csv(r'../cities_output_student.csv')
cities_df.head()


,Cities,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,Date
0,Thompson,55.7435,-97.8558,69.96,56,75,4.61,CA,2021-07-28 01:55:12
1,Rikitea,-23.1203,-134.9692,74.03,78,100,15.28,PF,2021-07-28 01:54:32
2,Ilulissat,69.2167,-51.1000,46.81,80,100,8.41,GL,2021-07-28 01:55:11
3,Kruisfontein,-34.0033,24.7314,53.89,84,35,14.23,ZA,2021-07-28 01:55:20
4,Clyde River,70.4692,-68.5914,46.63,81,90,8.05,CA,2021-07-28 01:56:55


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [26]:
# Configure gmaps
gmaps.configure(api_key = g_key)

# Store lat and lng as locations
locations = cities_df[['Latitude','Longitude']]

# Store humidity as weight
weight = cities_df['Humidity'].astype(float)

# Plot Heatmap
fig =gmaps.figure()

# Creat heat layer
heat_layer = gmaps.heatmap_layer(locations, weights = weight, dissipating = False, max_intensity=100, point_radius=3)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [27]:
# Creat new DataFrame fitting ideal conditions
vacation_spot_df = cities_df.loc[(cities_df['Max Temperature'] > 70) & (cities_df['Max Temperature'] < 80) & (cities_df['Wind Speed'] < 10) & (cities_df['Cloudiness'] < 5)].dropna()
vacation_spot_df

,Cities,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,Date
27,Jamestown,42.0970,-79.2353,71.24,78,1,5.12,US,2021-07-28 01:55:06
60,Moranbah,-22.0016,148.0466,74.79,38,4,5.30,AU,2021-07-28 01:58:07
73,Camopi,3.1655,-52.3158,71.22,99,2,2.75,GF,2021-07-28 01:58:09
99,Saint Lawrence,36.0553,14.2040,78.85,77,0,4.70,MT,2021-07-28 01:58:13
120,Belmonte,-15.8631,-38.8828,72.10,86,3,8.39,BR,2021-07-28 01:57:10
157,Mitsamiouli,-11.3847,43.2844,75.61,68,0,7.92,KM,2021-07-28 01:58:25
207,Sarkand,45.4100,79.9186,70.23,44,0,6.33,KZ,2021-07-28 01:58:35
236,Jacareacanga,-6.2222,-57.7528,75.58,48,2,0.69,BR,2021-07-28 01:57:16
245,Port Macquarie,-31.4333,152.9167,74.89,37,1,9.69,AU,2021-07-28 01:55:12


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [28]:
# Store variable named hotel_df
hotel_df = vacation_spot_df

# Add Hotel Name column
hotel_df["Hotel Name"] = ""

# Define parameters
params = {
    "radius": 5000,
    "types" : "hotel",
    "keyword" : "hotels",
    "key" : g_key
}

# Loop through API
for index, row in hotel_df.iterrows():
    
    # Define latitudes and longitudes
    lat = row["Latitude"]
    lng = row["Longitude"]
    
    # Maintain parameters with various locations
    params["location"] = f"{lat},{lng}"
    
    # Define url
    main_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # Capture hotel name
    hotel_name = requests.get(main_url, params = params)
    
    # Convert to json file
    hotel_name = hotel_name.json()
#     print(json.dumps(hotel_name, indent=4, sort_keys=True))
    
    # Include option to skip missing data
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_name["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing result.... Skipping.")
        
hotel_df

Missing result.... Skipping.


,Cities,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
27,Jamestown,42.0970,-79.2353,71.24,78,1,5.12,US,2021-07-28 01:55:06,Chautauqua Harbor Hotel
60,Moranbah,-22.0016,148.0466,74.79,38,4,5.30,AU,2021-07-28 01:58:07,Oaks Moranbah Suites
73,Camopi,3.1655,-52.3158,71.22,99,2,2.75,GF,2021-07-28 01:58:09,
99,Saint Lawrence,36.0553,14.2040,78.85,77,0,4.70,MT,2021-07-28 01:58:13,Saint Patrick's Hotel Gozo
120,Belmonte,-15.8631,-38.8828,72.10,86,3,8.39,BR,2021-07-28 01:57:10,Pousada do Diogo
157,Mitsamiouli,-11.3847,43.2844,75.61,68,0,7.92,KM,2021-07-28 01:58:25,Al Camar Lodge
207,Sarkand,45.4100,79.9186,70.23,44,0,6.33,KZ,2021-07-28 01:58:35,Гостиница Сарканд
236,Jacareacanga,-6.2222,-57.7528,75.58,48,2,0.69,BR,2021-07-28 01:57:16,HOTEL VIANA
245,Port Macquarie,-31.4333,152.9167,74.89,37,1,9.69,AU,2021-07-28 01:55:12,Sails Port Macquarie by Rydges


In [29]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{Cities}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [30]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))